In [ ]:
# default_exp scraper

# The Guardian Scraper

> Scraping Premier League Previews from the Guardian.

<div style="font-size: 200px">
    
|            Issues                 |          Solutions          |
|------------------------------     |-------------------|
|   4 possible formats for previews(old format, new format,Cup's format and a particular format) |Select the appropriate html tags|
|   Preview titles are not the same ( we can find Squad Sheets or match preview)|Pick only the names of the teams and eliminate the rest|
|   The date of the match is not always available |Pick the preview date|
|   The order of the elements and labels are not the same |Using regex patterns to get information|
|   Missing values for betting odds |We treat the general case separately and we set up specific regex patterns for these particular cases|
|   Odds format is different|We treat the general case separately and we set up specific regex patterns for these particular cases|
|   We can find non-numeric values for Odds like (Evens,evens,Eve)|Replace evens by 1-1|
|   There are some previews that don't have author and text|For previews that have no text, we put None (not available)|
|   The existence of previews for the FA CUP,Carabao Cup,Champions league,World Cup|Filter previews by title,link,topic,aside html section and preview text and allow only Premier League previews|
|   We are not sure if the names of the teams are the same as the ones in Opta|Set up a dictionary or check manually to map teams to their IDs|
|When we send many requests, the guardian server blocks your IP address, which is interpreted as a DDOS attack|Do a sleep of a random x seconds between requests or change your IP and work with rotating proxy|
</div >


### Import Libraries and Modules

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import pandas as pd
import dateparser
import random
import requests
import logging
from guardian_scraper.Db_Config import *
from guardian_scraper.parser import *
from guardian_scraper.extractor import *
from guardian_scraper.mapper import *
from guardian_scraper.models.preview import *
from typing import Dict, Union
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime

### ScrapingTheGuardian Class

##### This class represents a scraper from the "Guardian" website and has 4 functions:

1- <b> calculate_betting_odds </b> returns decimal odds.

&emsp;In this section, we will calculate the odds derived from the football preview.
<br>&emsp;Considering the following example:
<br>&emsp;&emsp; ["9-20","29-5","6-5"] 
<br>&emsp;&emsp;We calculate each sport's rating separately using the following formula:
<br>&emsp;&emsp;&emsp; home = (9/20) + 1 
<br>&emsp;&emsp;&emsp; away = (29/5) + 1
<br>&emsp;&emsp;&emsp; draw = (6/5) + 1
<br>&emsp;If we were successful in obtaining decimal odds, they will be returned in a Python dictionary.<br>&emsp;Otherwise, the values will be None (Not available).


2- <b> extract_preview_items </b>returns the entire contents of a football preview.

&emsp;In this section, we will call the functions defined in the PageExtractor class and return a Python dictionary containing all of this information.
<br>&emsp;But first, we use the <b>calculate_betting_odds</b> function to calculate the sports odds for the home team's victory, the away team's victory, and a draw.

"game Id","home team","away team","text","author","venue","referee","odds","odds home team","odds away team","odds draw", "preview date","game Date","preview_link" are the returned values.

3- <b> extract_previews </b> returns the information of all extracted previews.

&emsp;We retrieve all previews for a given page and go through them one by one, taking the link and getting its data using the "Guardian Api."
<br>&emsp;If the "Guardian Api" does not work, we will resort to the traditional process of html parsing. 
<br>&emsp;For each preview in the data extracted from the Guardian, we will look for the id of the home team and the away team and match it with the <b>opta.fixture</b><br>&emsp;database to get the gameID and gameDate and finally we save it in a MongoDb collection.
<br>&emsp;If the game exists, we will proceed with the data extraction.
The previous function, <b>extract_preview_items</b>, will be called here to extract information from <br>&emsp;each 
preview, which will then be stored in the list "all previews."
<br>&emsp;However, we will only extract previews that do not already exist in the database.

In [ ]:
# export
class ScrapingTheGuardian:
    """
    A class to represent a scraper from the "Guardian" website.

    ...

    Attributes
    ----------
    session : requests_html.HTMLSession
        a web session
    VENUE_REGEX : str
        venue regex expression
    REFEREE_REGEX : str
        referee regex expression
    ODDS_REGEX : str
        odds regex expression

    Methods
    -------
    calculate_betting_odds(odds)
        returns decimal odds.
    extract_preview_items(content,link,preview_date,game_date,game_id,home_team,away_team)
        returns all information of a football preview.
    extract_previews(self,page,previews_last_date,last_preview,all_previews,df_teams)
        returns the information of all extracted previews.

    """

    # venue, referee, odds pattern regex
    # in some previews, all of the information is on the same line.
    VENUE_REGEX = "Venue(.*)Tickets|Venue(.*),|Venue(.*)"
    REFEREE_REGEX = "Referee(.*)This season|Referee(.*)Last season's|Referee(.*)Odds|Referee[\s](.*)|Ref(.*)Odds"
    # {Odds H 11-8 A 11-8 D 11-8}
    # {Odds Liverpool 11-8 Aston Villa 11-8 Draw 11-8}
    # missing label {Odds H 11-8 11-8 D 11-8}
    # missing value {Odds H 11-8 A 11-8}
    ODDS_REGEX = "Odds[\s]*[a-zA-Z' ]*(\d{1,3}-[\s]*\d{1,3})[\s]*[a-zA-Z' ]*(\d{1,3}-[\s]*\d{1,3})([\s]*[a-zA-Z']*[\s]*(\d{1,3}-[\s]*\d{1,3}))*"

    def __init__(self):

        # Initialize session to start scraping
        self.session = HTMLSession()

    @staticmethod
    def calculate_betting_odds(odds: list) -> Dict[str, object]:
        """
          returns decimal odds.

        Parameters
        ----------
        odds: list of str
            odds values

        Returns
        -------
        betting_odds: dict of object

        """
        # Initialize betting odds to n/a (not available)
        # Some previews may not include odds
        odds_home = None
        odds_away = None
        odds_draw = None

        if odds is not None:  # If odds exist
            # example of odds:
            # {H 4-6 A 43-10 D 3-1}
            # {liverpool 4-6 Tottenham 43-10 Draw 3-1}
            # {H 4-6 43-10 D 3-1}
            # {H 4-6 A 43-10}
            # The formula will be (4/6)+1 , (43/10)+1 , (3/1)+1
            # Home team odds
            betting_odds_home = odds[0]
            try:
                odds_home = (
                    int(betting_odds_home.split("-")[0])
                    / int(betting_odds_home.split("-")[1])
                ) + 1
            except ZeroDivisionError:
                logging.error("Home team odds are wrong")
                pass
            # Away team odds
            betting_odds_away = odds[1]
            try:
                odds_away = (
                    int(betting_odds_away.split("-")[0])
                    / int(betting_odds_away.split("-")[1])
                ) + 1
            except ZeroDivisionError:
                logging.error("Away team odds are wrong")
                pass
            # if we have the normal format of odds
            # we will have 3 parts(odds_home,odds_away,odds_draw)
            if len(odds) >= 3:
                odds.pop(2)
                # Draw odds
                betting_odds_draw = odds[2]
                try:
                    odds_draw = (
                        int(betting_odds_draw.split("-")[0])
                        / int(betting_odds_draw.split("-")[1])
                    ) + 1
                except ZeroDivisionError:
                    logging.error("Draw odds are wrong")
                    pass

        betting_odds = dict(
            {"odds_home": odds_home, "odds_away": odds_away, "odds_draw": odds_draw}
        )
        return betting_odds

    @staticmethod
    def extract_preview_items(
        content: BeautifulSoup,
        link: str,
        preview_date: datetime,
        game_date: datetime,
        game_id: int,
        home_team: str,
        away_team: str,
        response_type: str,
    ) -> Dict[str, object]:
        """
          returns all information of a football preview

        Parameters
        ----------
        content: bs4.BeautifulSoup
            the html format of the preview content
        link: str
            the link of the preview
        preview_date: datetime
            the preview date
        game_date: datetime
            the game date
        game_id: int
            the game id
        home_team: str
            the home team name
        away_team: str
            the away team name
        response_type: str
            the parsing method('api' or 'html')

        Returns
        -------
        preview_items: dict of object

        """

        # meth1: extract match infos (venue,referee,odds)
        match_infos = PageExtractor.extract_match_infos(
            content,
            response_type,
            ScrapingTheGuardian.VENUE_REGEX,
            ScrapingTheGuardian.REFEREE_REGEX,
            ScrapingTheGuardian.ODDS_REGEX,
        )
        venue = match_infos["venue"]
        referee = match_infos["referee"]
        odds = match_infos["odds"]
        # meth2: extract text and author of the preview
        text_author = PageExtractor.extract_text_authors(content, response_type)
        text = text_author["text"]
        author = text_author["author"]
        # meth3: calculate betting odds
        betting_odds = ScrapingTheGuardian.calculate_betting_odds(odds)
        # Home team betting odds
        odds_home_team = betting_odds["odds_home"]
        # Away team betting odds
        odds_away_team = betting_odds["odds_away"]
        # Draw betting odds
        odds_draw = betting_odds["odds_draw"]
        # Return preview items
        preview_items = dict(
            {
                "game_id": game_id,
                "home_team": home_team,
                "away_team": away_team,
                "text": text,
                "author": author,
                "venue": venue,
                "referee": referee,
                "odds": odds,
                "odds_home_team": odds_home_team,
                "odds_away_team": odds_away_team,
                "odds_draw": odds_draw,
                "preview_date": preview_date,
                "game_date": game_date,
                "preview_link": link,
            }
        )
        return preview_items

    def extract_previews(
        self,
        page: BeautifulSoup,
        previews_last_date: datetime,
        last_preview: bool,
        all_previews: list,
        df_teams: pd.DataFrame,
    ) -> Union[bool, list]:
        """
          save all browsed previews in local

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page
        previews_last_date : datetime
            the last extracted preview date in the database
        last_preview: bool
            an indicator to know when we should stop the scraper
        all_previews: list
            a list that contains all extracted previews
        df_teams: pd.DataFrame
            a dataframe that contains teams and their different names

        Returns
        -------
        bool

        """
        # We pick all of the match previews on the webpage.
        previews = page.findAll("div", {"class": "fc-item__content"})
        # for each preview we extract its information.
        for preview in previews:
            # we pick the preview date and we parse it in a date format
            preview_date = preview.find("time")["datetime"]
            preview_date = dateparser.parse(preview_date, settings={"TIMEZONE": "UTC"})
            # if the date selected from the previews database exists
            # and has been reached by the preview date, we stop the loop
            # and mark last_preview as True.
            if previews_last_date and preview_date.date() <= previews_last_date.date():
                logging.info("The scraper turned off")
                last_preview = True
                break
            # Pick the preview link
            preview_link = preview.find("a")["href"]

            # We extract the last part of the link, which corresponds to the preview api link
            api_preview_url = preview_link.replace("https://www.theguardian.com", "")
            # request the api
            response = requests.get(
                "https://content.guardianapis.com/"
                + api_preview_url
                + "?api-key=fd4452e9-76a5-45a1-b30d-bdd156640b9c&show-blocks=all"
            )
            # if the api works we get the title and the content of the preview
            # else we extract html contents
            if response:
                logging.info("The Guardian Api works")
                # get the preview data
                data = response.json()
                # preview title
                preview_title = data["response"]["content"]["webTitle"]
                # preview content
                preview_content = BeautifulSoup(
                    data["response"]["content"]["blocks"]["body"][0]["bodyHtml"],
                    "html.parser",
                )
                # preview date
                preview_date = data["response"]["content"]["webPublicationDate"]
                preview_date = dateparser.parse(
                    preview_date, settings={"TIMEZONE": "UTC"}
                )
                response_type = "api"

            else:
                logging.info("The Guardian Api does not work")
                preview_content = Parser.parse_page(preview_link, self.session)
                preview_title = preview_content.find("h1").text
                response_type = "html"

            # extract team names
            names = PageExtractor.extract_teams_names(preview_title)
            # Home team and  Away Team
            home_team = names["home"]
            away_team = names["away"]
            # get teams id
            home_team_id = PreviewsMapping.get_team_id(home_team, df_teams)
            away_team_id = PreviewsMapping.get_team_id(away_team, df_teams)
            # pick the preview date
            # get the id and the date of the game
            game = PreviewsMapping.get_game_id_date(
                home_team_id, away_team_id, preview_date
            )
            # if the game exists we extract the preview information
            if game != None:
                preview_infos = ScrapingTheGuardian.extract_preview_items(
                    preview_content,
                    preview_link,
                    preview_date,
                    game.gameDate,
                    game.gameId,
                    home_team,
                    away_team,
                    response_type,
                )
                logging.info("Returned Preview information: {}".format(preview_infos))
                # connect to database
                mongoengine_client = MongoClient.connect("1")
                # preview class
                preview = Previews(
                    gameId=preview_infos["game_id"],
                    homeTeam=preview_infos["home_team"],
                    awayTeam=preview_infos["away_team"],
                    text=preview_infos["text"],
                    author=preview_infos["author"],
                    venue=preview_infos["venue"],
                    referee=preview_infos["referee"],
                    odds=preview_infos["odds"],
                    oddsHomeTeam=preview_infos["odds_home_team"],
                    oddsAwayTeam=preview_infos["odds_away_team"],
                    oddsDraw=preview_infos["odds_draw"],
                    gameDate=preview_infos["game_date"],
                    previewDate=preview_infos["preview_date"],
                    previewLink=preview_infos["preview_link"],
                )
                # Validate and save input raw data
                MongoClient.save(preview)
                all_previews.append(preview_infos)

            else:
                logging.info(
                    "The game {} does not exist in the Opta database".format(
                        preview_title
                    )
                )

        return last_preview, all_previews

### Scraping all pages

In [ ]:
# store scraper actions in a log file
logging.basicConfig(
    filename="scraper.log", level=logging.INFO, format="%(levelname)s:%(message)s"
)
# starting url
url = "https://www.theguardian.com/football/series/match-previews"
# initialize the scraper instance.
scraper = ScrapingTheGuardian()
# initially we are not at the last page.
last_page = False
# we'll extract the previews that haven't already been extracted.
last_preview = False
# we specify the last preview date in the previews collection on which the scraper will be turned off.
mongoengine_client = MongoClient.connect("1")
previews_last_date = Previews.objects().order_by("-previewDate", "-gameDate").first()

# if the database is empty we will scrap all pages
if previews_last_date != None:
    last_previews_date = previews_last_date.previewDate
else:
    last_previews_date = None

logging.info("The last preview date stored in the database is: {}".format(last_previews_date))
all_previews = []
# charging teams dictionary
df_teams = pd.read_csv(".//datasets//final_data.csv")
# if we are not at the last page
# and we haven't reached an extracted preview
# we launch the scraper
while not last_page and not last_preview:
    # a random timer
    time = random.randint(2, 60)
    logging.info("The scraper will wait {} seconds ...".format(time))
    logging.info("The scraper will open this url: ".format(url))
    # wait time seconds
    sleep(time)
    # get the html format of the page containing previews
    page = Parser.parse_page(url, scraper.session)
    # launch the scraper , extract previews information
    last_preview, all_previews = scraper.extract_previews(
        page, last_previews_date, last_preview, all_previews, df_teams
    )
    # get the url of the following page and verify if we are at the last page
    url, last_page = Parser.get_next_page(page)

19
https://www.theguardian.com/football/series/match-previews?page=42
https://www.theguardian.com/football/series/match-previews?page=42
https://www.theguardian.com/football/2019/jan/19/huddersfield-manchester-city-match-preview-premier-league
{'game_id': 987815, 'home_team': 'Huddersfield', 'away_team': 'Manchester City', 'text': 'Most Huddersfield fans were sad to see David Wagner leave as manager despite the club’s position at the bottom of the Premier League. Beating Manchester City while under the caretaker charge of Mark Hudson would go some way to cheering up the locals and convincing them that relegation is not a certainty. But it is a mighty tall order, with Pep Guardiola’s side in fabulous form as they continue to chase Liverpool at the top of the table. Paul Doyle', 'author': 'Paul Doyle', 'venue': 'John Smith’s Stadium', 'referee': 'Andre Marriner', 'odds': ['22-1', '1-6', '8-1'], 'odds_home_team': 23.0, 'odds_away_team': 1.1666666666666667, 'odds_draw': 9.0, 'preview_date'

https://www.theguardian.com/football/2019/jan/18/southampton-everton-match-preview
{'game_id': 987819, 'home_team': 'Southampton', 'away_team': 'Everton', 'text': 'Southampton could further ease their relegation fears by following up victory at Leicester against another top-half team struggling to meet high expectations. Ralph Hasenhüttl was left concerned by the midweek FA Cup defeat on penalties to Derby, which has sapped his squad’s energy as well as their morale. The Austrian plans to freshen up his starting XI, with Danny Ings winning his fitness battle and both Charlie Austin and Pierre-Emile Højbjerg back from suspension. Everton’s away form this season and record at St Mary’s are equally poor – two wins from 10 in both cases. Marco Silva has suggested he may not invest in attacking reinforcements in this transfer window, and is banking on Richarlison rediscovering his goalscoring touch – the Brazilian last scored on Boxing Day. Niall McVeigh', 'author': 'Niall McVeigh', 'venue'

https://www.theguardian.com/football/2019/jan/11/crystal-palace-watford-match-preview
{'game_id': 987806, 'home_team': 'Crystal Palace', 'away_team': 'Watford', 'text': 'Watford have been a rollercoaster ride producing the sublime and less sublime so far. They have been an open book scoring and conceding freely, the 3-3 draw with Bournemouth last time out a perfect example of their season. Crystal Palace are enjoying a welcome upturn with unlikely wins against Leicester, Manchester City and Wolves in their last five matches. Another win could put Palace nine points above the relegation zone. Javi Gracia should be worried, especially as the sole goalscoring burden has been lifted from Wilfired Zaha. Graham Searles', 'author': 'Graham Searles', 'venue': 'Selhurst Park', 'referee': 'Paul Tierney', 'odds': ['23-18', '5-2', '23-10'], 'odds_home_team': 2.2777777777777777, 'odds_away_team': 3.5, 'odds_draw': 3.3, 'preview_date': datetime.datetime(2019, 1, 11, 14, 23, 16, tzinfo=<StaticTzInfo 

https://www.theguardian.com/football/2018/dec/28/tottenham-wolves-match-preview
{'game_id': 987790, 'home_team': 'Tottenham', 'away_team': 'Wolverhampton Wanderers', 'text': 'Mauricio Pochettino can reflect on the first-half of the season with pride. Flying high in the Premier League, his Tottenham side are also into the Champions League last 16 and the Carabao Cup semi-finals. He could scarcely have wished for more yet it is what lies ahead that tantalises. Can Spurs win the league? Play like they have done since Pochettino was strongly linked to Manchester United and they could. Wolves have stabilised after a wobble and they will threaten on the counter. David Hytner', 'author': 'David Hytner', 'venue': 'Wembley Stadium', 'referee': 'Stuart Atwell', 'odds': ['1-3', '9-1', '4-1'], 'odds_home_team': 1.3333333333333333, 'odds_away_team': 10.0, 'odds_draw': 5.0, 'preview_date': datetime.datetime(2018, 12, 28, 16, 45, 59, tzinfo=<StaticTzInfo 'UTC'>), 'game_date': datetime.datetime(2018, 

https://www.theguardian.com/football/2018/dec/21/cardiff-manchester-united-match-preview
{'game_id': 987764, 'home_team': 'Cardiff', 'away_team': 'Manchester United', 'text': 'In a strange twist of fate, Ole Gunnar Solskjær’s first match in charge of Manchester United is back at the club where he was sacked after only nine months in charge. That was in 2014 and few Cardiff fans could have imagined the Norwegian would return one day as the manager of United. Although this is an awkward game for Solskjær – Cardiff have won four of their past five at home – there is a sense that United’s players will feel liberated now that José Mourinho has gone. Stuart James', 'author': 'Stuart James', 'venue': 'Cardiff City Stadium', 'referee': 'Michael Oliver', 'odds': ['11-2', '3-5', '3-1'], 'odds_home_team': 6.5, 'odds_away_team': 1.6, 'odds_draw': 4.0, 'preview_date': datetime.datetime(2018, 12, 21, 16, 25, 56, tzinfo=<StaticTzInfo 'UTC'>), 'game_date': datetime.datetime(2018, 12, 22, 17, 30), 'pre

...

https://www.theguardian.com/football/2009/sep/25/squahd-sheets-fulham-arsenal
{'game_id': 284062, 'home_team': 'Fulham', 'away_team': 'Arsenal', 'text': "Arsenal will hope to continue their rich vein of form in front of goal this weekend, but face a tough test against Roy Hodgson's well-drilled back-line. Fulham's defensive record so far this season is better than that of the visitors, although a paltry four league goals – less than a quarter of what Arsenal have mustered – must be a real cause for concern. With Vito Mannone expected to remain in goal for Arsène Wenger's side, however, Fulham's misfiring forwards will sense an opportunity to bolster their tallies. Lee Price", 'author': 'Lee Price', 'venue': 'Craven Cottage', 'referee': 'M Atkinson', 'odds': ['9-2', '11-20', '13-5'], 'odds_home_team': 5.5, 'odds_away_team': 1.55, 'odds_draw': 3.6, 'preview_date': datetime.datetime(2009, 9, 25, 15, 37, 55, tzinfo=<StaticTzInfo 'UTC'>), 'game_date': datetime.datetime(2009, 9, 26, 16, 30),

https://www.theguardian.com/football/2009/sep/18/squad-sheets-chelsea-tottenham-hotspur
{'game_id': 284054, 'home_team': 'Chelsea', 'away_team': 'Tottenham Hotspur', 'text': "When Jermain Defoe leapt to volley Tottenham into a first-minute lead against Manchester United last weekend, the dreams of their supporters ran wild. Their flawless start to the season would add the scalp of the champions to that of Liverpool. United's subsequent comeback supplied the reality check and now comes the trip to a stadium where they have not won in 19 years and 22 matches. Spurs will attack but Chelsea, relentless and remorseless, have yet to be stopped under their latest manager, Carlo Ancelotti. David Hytner", 'author': 'David Hytner', 'venue': 'Stamford Bridge', 'referee': 'H Webb', 'odds': ['9-20', '11-2', '29-10'], 'odds_home_team': 1.45, 'odds_away_team': 6.5, 'odds_draw': 3.9, 'preview_date': datetime.datetime(2009, 9, 18, 16, 15, 19, tzinfo=<StaticTzInfo 'UTC'>), 'game_date': datetime.datetime

https://www.theguardian.com/football/2009/sep/11/squad-sheets-everton-fulham
{'game_id': 284042, 'home_team': 'Fulham', 'away_team': 'Everton', 'text': 'Fulham are the only club in the Premier League who can point out that their season started in July, with a Europa League qualifying tie away to Vetra, and one of the greatest challenges facing Roy Hodgson is accommodating the demands of Europe. His team looked jaded at Aston Villa two weekends ago after the long trip to play Amkar Perm in eastern Russia. The Everton manager, David Moyes, has similar worries, after his team also advanced to the Europa League group phase, but at least this is more familiar territory to his squad. David Hytner', 'author': 'David Hytner', 'venue': 'Craven Cottage', 'referee': 'P Walton', 'odds': ['13-10', '9-5', '11-5'], 'odds_home_team': 2.3, 'odds_away_team': 2.8, 'odds_draw': 3.2, 'preview_date': datetime.datetime(2009, 9, 11, 19, 52, 47, tzinfo=<StaticTzInfo 'UTC'>), 'game_date': datetime.datetime(2009

https://www.theguardian.com/football/2009/aug/21/squad-sheets-west-ham-tottenham
{'game_id': 284028, 'home_team': 'West Ham United', 'away_team': 'Tottenham Hotspur', 'text': 'The current league leaders step out into the feverish atmosphere generated by this fixture attempting to maintain rare momentum. It took Tottenham 10 games last season to muster the six points they boast from two fixtures this time around, and eight, seven and six matches in the three years before. The midweek thumping of Hull City sharpened Jermain Defoe up for a return to his former club though West Ham, impressive in defeating Wolves and a slick threat under Gianfranco Zola, will provide a sterner test of their mettle. Dominic Fifield', 'author': 'Dominic Fifield', 'venue': 'Upton Park', 'referee': 'M Clattenburg', 'odds': ['9-5', '5-4', '23-10'], 'odds_home_team': 2.8, 'odds_away_team': 2.25, 'odds_draw': 3.3, 'preview_date': datetime.datetime(2009, 8, 21, 21, 50, tzinfo=<StaticTzInfo 'UTC'>), 'game_date': da

https://www.theguardian.com/football/2009/aug/21/squad-sheets-sunderland-blackburn-rovers
{'game_id': 284027, 'home_team': 'Sunderland', 'away_team': 'Blackburn Rovers', 'text': 'As a former Newcastle United manager, Blackburn\'s Sam Allardyce can expect a hot reception. Steve Bruce, meanwhile, anticipates a somewhat gruelling afternoon. "Sam is very, very well organised and his teams play to a system which is not always easy on the eye but is very effective," said Sunderland\'s manager. "But these are the games we are going to be judged on." This is the home side\'s third match in a week and, with injuries mounting, it will be intriguing to see if Bruce offers fit crowd favourite Andy Reid a start. Louise Taylor', 'author': 'Louise Taylor', 'venue': 'Stadium of Light, Saturday', 'referee': 'A Wiley', 'odds': ['1-1', '12-5', '9-4'], 'odds_home_team': 2.0, 'odds_away_team': 3.4, 'odds_draw': 3.25, 'preview_date': datetime.datetime(2009, 8, 21, 21, 8, 35, tzinfo=<StaticTzInfo 'UTC'>), 'g

In [ ]:
data = pd.DataFrame(all_previews)

In [ ]:
data

,game_id,home_team,away_team,text,author,venue,referee,odds,odds_home_team,odds_away_team,odds_draw,preview_date,game_date,preview_link
0,987815,Huddersfield,Manchester City,Most Huddersfield fans were sad to see David W...,Paul Doyle,John Smith’s Stadium,Andre Marriner,"[22-1, 1-6, 8-1]",23.000000,1.166667,9.00,2019-01-19 09:00:12+00:00,2019-01-20 13:30:00,https://www.theguardian.com/football/2019/jan/...
1,987818,Newcastle United,Cardiff,There is little love lost between Rafa Benítez...,Louise Taylor,St James’ Park,Stuart Atwell,"[1-1, 10-3, 12-5]",2.000000,4.333333,3.40,2019-01-18 16:57:35+00:00,2019-01-19 15:00:00,https://www.theguardian.com/football/2019/jan/...
2,987821,Wolverhampton Wanderers,Leicester,Wolves have lost five of their last seven matc...,Graham Searles,Molineux,Chris Kavanagh,"[6-5, 5-2, 2-1]",2.200000,3.500000,3.00,2019-01-18 16:19:44+00:00,2019-01-19 12:30:00,https://www.theguardian.com/football/2019/jan/...
3,987820,Watford,Burnley,Watford and Burnley are unbeaten since Boxing ...,Graham Searles,Vicarage Road,Michael Oliver,"[4-6, 5-1, 3-1]",1.666667,6.000000,4.00,2019-01-18 15:15:04+00:00,2019-01-19 15:00:00,https://www.theguardian.com/football/2019/jan/...
4,987817,Manchester United,Brighton,If Manchester United can reel off the seventh ...,Jamie Jackson,Old Trafford,Paul Tierney,"[1-3, 12-1, 5-1]",1.333333,13.000000,6.00,2019-01-18 14:23:22+00:00,2019-01-19 15:00:00,https://www.theguardian.com/football/2019/jan/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,284023,Fulham,Chelsea,"The achievements of Fulham, who came seventh l...",Kevin McCarra,Craven Cottage,A Marriner,"[9-2, 4-7, 5-2]",5.500000,1.571429,3.50,2009-08-21 21:24:27+00:00,2009-08-23 15:00:00,https://www.theguardian.com/football/2009/aug/...
3064,284022,Burnley,Everton,Turf Moor thrived on its first top-flight game...,Andy Hunter,"Turf Moor, Saturday",P Dowd,"[2-1, 6-5, 11-5]",3.000000,2.200000,3.20,2009-08-21 21:21:25+00:00,2009-08-23 14:00:00,https://www.theguardian.com/football/2009/aug/...
3065,284021,Birmingham City,Stoke City,If Birmingham are to stay in the Premier Leagu...,David Lacey,"St Andrew's, Saturday",C Foy,"[23-20, 21-10, 11-5]",2.150000,3.100000,3.20,2009-08-21 21:15:59+00:00,2009-08-22 14:00:00,https://www.theguardian.com/football/2009/aug/...
3066,284020,Arsenal,Portsmouth,Peter Storrie says he has a mystery new backer...,David Hytner,"Emirates Stadium, Saturday",S Bennett,"[1-6, 12-1, 5-1]",1.166667,13.000000,6.00,2009-08-21 21:13:41+00:00,2009-08-22 14:00:00,https://www.theguardian.com/football/2009/aug/...
